In [14]:
from tensorflow.keras.layers import Input,Lambda,Dense,Flatten
from tensorflow.keras.models import Model,Sequential,load_model
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
import  numpy as np
from  glob import glob




In [3]:
IMAGE_SIZE = [224,224]

train_path = 'Datasets/train'
test_path = 'Datasets/test'

In [4]:
resnet = ResNet50(input_shape=IMAGE_SIZE + [3],weights='imagenet',include_top=False)


In [5]:
for layer in resnet.layers:
    layer.trainable = False

In [6]:
folders = glob('Datasets/Train/*')
print(folders)
x= Flatten()(resnet.output)

['Datasets/Train/audi', 'Datasets/Train/mercedes', 'Datasets/Train/lamborghini']


In [7]:
prediction = Dense((len(folders)),activation='softmax')(x)
model = Model(inputs=resnet.input,outputs=prediction)

In [8]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [9]:
train_data = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip= True)

test_data = ImageDataGenerator(rescale=1./255)

training_set = train_data.flow_from_directory('Datasets/Train',target_size=(224,224),batch_size=32,class_mode='categorical')

test_set = test_data.flow_from_directory('Datasets/Test',target_size=(224,224),batch_size=32,class_mode='categorical')

Found 64 images belonging to 3 classes.
Found 58 images belonging to 3 classes.


In [ ]:
r = model.fit_generator(training_set,validation_data=test_set,epochs=100,steps_per_epoch=len(training_set),validation_steps=(len(test_set)))


In [11]:
model.save('model_resnet.h5')

In [12]:
y_pred = model.predict(test_set)
#print(y_pred)

y_pred = np.argmax(y_pred,axis=1)

#print(y_pred)

In [15]:
model = load_model('model_resnet.h5')

img = image.load_img('Datasets/Test/lamborghini/10.jpg',target_size=(224,224))

x = image.img_to_array(img)

In [16]:
x=x/225

x = np.expand_dims(x,axis=0)
img = preprocess_input(x)
print(img.shape)

(1, 224, 224, 3)


In [17]:
model.predict(img)

a=np.argmax(model.predict(img),axis=1)

print(a)

[2]
